In [1]:
pip install ollama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import ollama

In [3]:
response=ollama.chat(model="deepseek-r1:1.5b",messages=[{"role":"user","content":"Hai"}])

In [4]:
response["message"]

Message(role='assistant', content='<think>\n\n</think>\n\nHello! How can I assist you today? 😊', images=None, tool_calls=None)

In [5]:
groq_api_key = "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"


In [6]:
pip install langchain langchain-community pdfplumber numpy scikit-learn faiss-cpu requests langchain-groq googlesearch-python beautifulsoup4 langchain-experimental sentence_transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from typing import List
from bs4 import BeautifulSoup
import requests
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.hyde.base import HypotheticalDocumentEmbedder
from langchain.llms import Ollama
# Monkey-patch with proper input_variables access
@property
def fixed_input_keys(self) -> List[str]:
    return self.llm_chain.prompt.input_variables  # Access through prompt

HypotheticalDocumentEmbedder.input_keys = fixed_input_keys

def get_retrievers(pdf_path, groq_api_key):
    import warnings
    warnings.filterwarnings("ignore")
    import random
    import pdfplumber
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity
    from langchain.embeddings import HuggingFaceBgeEmbeddings
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain_community.document_loaders import PyPDFLoader
    from langchain.docstore.document import Document
    from langchain_community.vectorstores import FAISS

    # Initialize HyDE components
    hyde_prompt = PromptTemplate(
        input_variables=["question"],
        template="""Generate a comprehensive hypothetical answer to: {question}
    Include key facts, concepts, and relevant context."""
    )

    hyde_llm = Ollama(
        model="deepseek-r1:1.5b"
    )
    hyde_chain = LLMChain(llm=hyde_llm, prompt=hyde_prompt)

    # Base embeddings model
    embedding_model = HuggingFaceBgeEmbeddings(
        model_name="BAAI/bge-large-en",
        encode_kwargs={'normalize_embeddings': False}
    )

    # Wrap with HyDE (using patched version)
    hyde_embeddings = HypotheticalDocumentEmbedder(
        llm_chain=hyde_chain,
        base_embeddings=embedding_model,
    )
    def embed_texts(texts):
        return embedding_model.embed_documents(texts)


    def get_header_footer(pdf_path, threshold=0.71):
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            if total_pages >= 15:
                random_page_nos = random.sample(range(5, total_pages), 10)
            else:
                random_page_nos = list(range(total_pages))

            avg_similarity = 1
            header_lines = -1
            while avg_similarity > threshold and header_lines < 4:
                header_lines += 1
                five_lines = []
                for page_no in random_page_nos:
                    lines = pdf.pages[page_no].extract_text().split('\n')
                    if len(lines) > header_lines:
                        five_lines.append(lines[header_lines])
                similarities = cosine_similarity(embed_texts(five_lines))
                avg_similarity = np.mean(similarities[np.triu_indices(len(similarities), k=1)])

            avg_similarity = 1
            footer_lines = -1
            while avg_similarity > threshold and footer_lines < 4:
                footer_lines += 1
                five_lines = []
                for page_no in random_page_nos:
                    lines = pdf.pages[page_no].extract_text().split('\n')
                    if len(lines) > footer_lines:
                        five_lines.append(lines[-(footer_lines + 1)])
                similarities = cosine_similarity(embed_texts(five_lines))
                avg_similarity = np.mean(similarities[np.triu_indices(len(similarities), k=1)])
            return header_lines, footer_lines


    def extract_text(pdf_path):
        header_lines, footer_lines = get_header_footer(pdf_path)
        with pdfplumber.open(pdf_path) as pdf:
            text = ''
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    lines = page_text.split('\n')
                    if lines:
                        page_text = '\n'.join(lines[header_lines:-(footer_lines + 1)])
                        text += page_text + '\n'
            return text

    text = extract_text(pdf_path)

    def get_vectorstore1():
        texts = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_text(text)
        docs = [Document(text) for text in texts if text.strip()]
        vectorstore = FAISS.from_documents(docs, hyde_embeddings)  # Use HyDE embeddings
        return vectorstore

    def get_vectorstore2():
        texts = RecursiveCharacterTextSplitter(chunk_size=6000, chunk_overlap=400).split_text(text)
        docs = [Document(text) for text in texts if text.strip()]
        vectorstore = FAISS.from_documents(docs, hyde_embeddings)  # Use HyDE embeddings
        return vectorstore

    retriever1 = get_vectorstore1().as_retriever(search_kwargs={"k": 6})
    retriever2 = get_vectorstore2().as_retriever(search_kwargs={"k": 3})
    return retriever1, retriever2

def web_search(query, max_results=3):
    """Perform web search using googlesearch-python"""
    from googlesearch import search
    results = list(search(query, num_results=max_results))
    return results[:max_results]

def fetch_content_from_link(link):
    try:
        if not link.startswith(('http://', 'https://')):
            link = f'https://{link}'
        response = requests.get(link, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        raw_text = soup.get_text()
        cleaned_text = ' '.join(raw_text.split())
        return cleaned_text
    except Exception as e:
        #print(f"Error fetching {link}: {str(e)}")
        return ""



def get_answer(query, retriever1, retriever2, groq_api_key):
    # Web search integration
    links = web_search(query)
    web_results = "\n".join([f"{i+1}. {fetch_content_from_link(link)}" for i, link in enumerate(links)])

    # HyDE-enhanced document retrieval
    doc_results1 = retriever1.get_relevant_documents(query)
    doc_results2 = retriever2.get_relevant_documents(query)
    doc_context = "\n---\n".join([doc.page_content for doc in doc_results1 + doc_results2])

    # Context management
    combined_context = f"""
    WEB SEARCH RESULTS:
    {web_results}

    DOCUMENT CONTENT:
    {doc_context}
    """
    if len(combined_context) > 4000:
        combined_context = combined_context[:4000]

    # LLM initialization
    llm = Ollama(
        model="deepseek-re:1.5b"
    )

    # Corrected prompt template
    prompt_template = """
    Analyze and synthesize information from both web results and document content to answer
    the question. Follow these steps:
    1. Identify key facts from web results.
    2. Find supporting information in documents.
    3. Combine insights from both sources.
    4. If sources conflict, note this and prioritize document content.
    5. Provide a clear, concise answer.
    6. Do not restate the question. Provide a direct comparison of the answers.
    7. Give a final judgment on which answer is better and why, without using phrases like 'based on web results' or unnecessary explanations.

    CONTEXT:
    {combined_context}

    QUESTION: {question}

    FINAL ANSWER:
    """
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["combined_context", "question"]  # Fixed placeholder reference
    )

    chain = LLMChain(llm=llm, prompt=prompt)

    return chain.run(combined_context=combined_context, question=query)  # Matching argument names


def compare_answers(query, retriever1, retriever2, retriever3, retriever4, groq_api_key):
    answer1 = get_answer(query, retriever1, retriever2, groq_api_key)
    answer2 = get_answer(query, retriever3, retriever4, groq_api_key)

    comparison_prompt = f"""
    Compare the two answers given for the same question:

    QUESTION: {query}

    ANSWER 1: {answer1}

    ANSWER 2: {answer2}

    Do not restate the question. Provide a direct comparison of the answers focusing only on:
    1. Factual consistency
    2. Source reliability
    3. Completeness of information
    4. Clarity of presentation

    Give a final judgment on which answer is better and why, without using phrases like 'based on web results' or unnecessary explanations.
    """


    llm = Ollama(
        model="deepseek-r1:1.5b"
    )

    return llm.invoke(comparison_prompt).content

In [9]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Define the prompt template
hyde_prompt = PromptTemplate(
    input_variables=["question"],
    template="""Generate a comprehensive hypothetical answer to: {question}
    Include key facts, concepts, and relevant context."""
)

# Use Ollama for local inference
hyde_llm = Ollama(model="deepseek-r1:1.5b")

# Create the chain
hyde_chain = LLMChain(llm=hyde_llm, prompt=hyde_prompt)

# Example usage
question = "What are the benefits of AI in healthcare?"
response = hyde_chain.run(question)
print(response)


<think>
Okay, so I need to figure out the benefits of AI in healthcare based on this detailed response from someone else. Let me read through it again.

First, they mentioned personalized treatment plans. That makes sense because AI can analyze a lot of data about a patient's symptoms and medical history. So, it tailors treatments to each individual's unique needs. But I wonder how exactly that works—like what kind of data is considered? Maybe things like genetic information or lifestyle factors?

Next, predicting diseases. They talk about early detection through machine learning models. That rings a bell. I've heard about breast cancer screening using AI. I should check if these predictions are accurate and when they can be used, maybe before the disease actually shows up.

Then, optimizing care delivery. So, AI helps plan treatment schedules. Maybe it uses AI to simulate different scenarios or recommend drugs that fit each patient's needs. This could make care more efficient by reduc